### Switch Models

This notebook provides basic functionality to switch models already trained. An assumption made is that
the model references in database and actual model files are in sync

- PS: Switching to DL model is buggy esp. after training was done recentlty as GPU memory gets hogged (atleast on my machine)

In [11]:
# # run this if running locally not requires if you used the docker script
# # !pip install python-dotenv
# from dotenv import load_dotenv
# load_dotenv(dotenv_path = '../.env')

In [12]:
import pandas as pd
#from mlcore.dbhelper import read_data
from mlcore.modelops import get_deployment_hist, get_deployed_model
from mlcore.dbhelper import store_data, overwrite_data
from mlcore.modelops import load_model

### See trained and deployed models

In [13]:
# Deployed model
deployed_model_name, deployed_model_obj = get_deployed_model(mldbpath='../data/mldbreg.sqlite')
print('Deployed Model is :::--> {}'.format(deployed_model_name))

2022-03-27 17:17:26,910:Loaded model stored at ../models/RF_model_03_27_2022_09_04_09.mdl
2022-03-27 17:17:26,910:Loaded model stored at ../models/RF_model_03_27_2022_09_04_09.mdl


Deployed Model is :::--> RF_model_03_27_2022_09_04_09


In [14]:
# Trained Models 
prev_trained_models = get_deployment_hist(mldbpath='../data/mldbreg.sqlite')
prev_trained_models

,index,final_model_name
0,0,RF_model_03_27_2022_09_04_09
1,0,RF_model_03_27_2022_17_08_31
2,0,RF_model_03_27_2022_09_04_09


### See trained and deployed models

In [15]:
# Helper Functions which help deploying a model
mldbpath = '../data/mldbreg.sqlite'

def update_model(deploy_df,mldbpath='../data/mldb.sqlite'):

    schema_dict = {
        'deployed_model':deploy_df,
        'hist_deployed_models':deploy_df,
        #'train_report':comparison_result
    }
    for dkey in schema_dict:
        data_to_be_stored = schema_dict[dkey]
        if dkey=='deployed_model':
             overwrite_data(data_to_be_stored, mldbpath, dkey)
        else:
            store_data(data_to_be_stored, mldbpath, dkey)

def switch_model(model_name, mldbpath='../data/mldb.sqlite'):
    deployed_model_name, deployed_model_obj = get_deployed_model(mldbpath=mldbpath)
    print('Deployed Model currently set to  :::--> {}'.format(deployed_model_name))
    if model_name==deployed_model_name:
        print('Model :::--> {} is already deployed'.format(deployed_model_name))
    
    else:
        new_model = load_model(model_name)
        
        deploy_df = pd.DataFrame([[model_name]], 
                                 columns =['final_model_name'])
        update_model(deploy_df,mldbpath=mldbpath)
        deployed_model_name, deployed_model_obj = get_deployed_model(mldbpath=mldbpath)
        print('Deployed Model updated to  :::--> {}'.format(model_name))
        


In [16]:
# Generates a log of models you can switch to
prev_trained_models_dict = prev_trained_models['final_model_name'].to_dict()

print('Your Can Change to below model names')
seen = set([deployed_model_name])
for m in prev_trained_models_dict:
    model_name = prev_trained_models_dict[m]
    if model_name not in seen:  
        print('Model Name:::--> {}'.format(model_name))
        seen.add(model_name)


Your Can Change to below model names
Model Name:::--> RF_model_03_27_2022_17_08_31


In [7]:
# # Uncomment Below and Switch based on above info
# model_to_switch_to = 'RF_model_03_27_2022_09_04_09'
# switch_model(model_to_switch_to,mldbpath='../data/mldbreg.sqlite')

2022-03-27 17:12:40,967:Loaded model stored at ../models/RF_model_03_27_2022_17_08_31.mdl
2022-03-27 17:12:40,967:Loaded model stored at ../models/RF_model_03_27_2022_17_08_31.mdl
2022-03-27 17:12:41,089:Loaded model stored at ../models/RF_model_03_27_2022_09_04_09.mdl
2022-03-27 17:12:41,089:Loaded model stored at ../models/RF_model_03_27_2022_09_04_09.mdl
2022-03-27 17:12:41,089:Loaded model stored at ../models/RF_model_03_27_2022_09_04_09.mdl
2022-03-27 17:12:41,162:Could not load deployed model it may not exist exception Execution failed on sql 'SELECT * from deployed_model': no such table: deployed_model


Deployed Model currently set to  :::--> RF_model_03_27_2022_17_08_31
Deployed Model updated to  :::--> RF_model_03_27_2022_09_04_09


In [8]:
prev_trained_models_dict

{0: 'RF_model_03_27_2022_09_04_09', 1: 'RF_model_03_27_2022_17_08_31'}